In [1]:
from config import *

In [2]:
levelups_ref = pd.read_csv("../data/levelups_panel_5.csv", encoding="utf-8")

In [3]:
levelups_ref[levelups_ref['total_purchase']>3300]

,player_slug,character_slug,first_session_level,total_purchase,total_ad,highest_level,status,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum,playtime_boss,playtime_dungeon,playtime_tower,playtime_pvp,playtime_farm,playtime_survival,playtime_side
1329,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,1,2,2021-11-01 22:54:11,0.0,0.0,0.0,0.0,0.0,2021-11-01 22:53:46,0.007222,0.007222,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1330,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,2,3,2021-11-01 22:54:23,0.0,0.0,0.0,0.0,0.0,2021-11-01 22:54:11,0.003611,0.010833,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1331,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,3,4,2021-11-01 22:54:54,0.0,0.0,0.0,0.0,0.0,2021-11-01 22:54:23,0.008889,0.019722,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1332,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,4,5,2021-11-01 22:55:28,0.0,0.0,0.0,0.0,0.0,2021-11-01 22:54:54,0.009722,0.029444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1333,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,5,6,2021-11-01 22:56:14,0.0,0.0,0.0,0.0,0.0,2021-11-01 22:55:28,0.013056,0.042500,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466261,2022-03-30T05:37:49.026Z,백야행,21,49500.0,0.0,30,complete,25,26,2022-03-30 18:18:53,0.0,0.0,0.0,0.0,0.0,2022-03-30 18:18:19,0.009722,3.649444,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466262,2022-03-30T05:37:49.026Z,백야행,21,49500.0,0.0,30,complete,26,27,2022-03-30 18:19:07,0.0,0.0,0.0,0.0,0.0,2022-03-30 18:18:53,0.004167,3.653611,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466263,2022-03-30T05:37:49.026Z,백야행,21,49500.0,0.0,30,complete,27,28,2022-03-30 18:19:28,0.0,0.0,0.0,0.0,0.0,2022-03-30 18:19:07,0.006111,3.659722,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2466264,2022-03-30T05:37:49.026Z,백야행,21,49500.0,0.0,30,complete,28,29,2022-03-30 18:20:04,0.0,0.0,0.0,0.0,0.0,2022-03-30 18:19:28,0.010278,3.670000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
currency_use= pd.read_csv("../data_raw/characters_currencies_use.csv", encoding="utf-8")
Dia_currency=currency_use[currency_use['currency_slug']=='다이아']
Gold_currency=currency_use[currency_use['currency_slug']=='골드']

In [5]:
levelups_ref['event_datetime'] = pd.to_datetime(levelups_ref['event_datetime'])
Gold_currency['event_datetime'] = pd.to_datetime(Gold_currency['event_datetime'])
Dia_currency['event_datetime'] = pd.to_datetime(Dia_currency['event_datetime'])

C:\Users\sunwoong\AppData\Local\Temp\ipykernel_34988\1700074789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Gold_currency['event_datetime'] = pd.to_datetime(Gold_currency['event_datetime'])
C:\Users\sunwoong\AppData\Local\Temp\ipykernel_34988\1700074789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dia_currency['event_datetime'] = pd.to_datetime(Dia_currency['event_datetime'])


In [9]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 업데이트할 새로운 열 초기화
levelups_ref['updated_gold_usage'] = 0

# 함수 정의: 각 인덱스에 대해 골드 사용량을 업데이트하는 함수
def update_gold_usage(i):
    start_time = levelups_ref.loc[i, 'event_datetime']
    end_time = levelups_ref.loc[i + 1, 'event_datetime']
    playerslug = levelups_ref.loc[i, 'player_slug']

    # 마스크 생성
    mask = (Gold_currency['event_datetime'] >= start_time) & \
           (Gold_currency['event_datetime'] < end_time) & \
           (Gold_currency['player_slug'] == playerslug)

    # 골드 사용량 계산
    updated_gold_usage = Gold_currency.loc[mask, 'currency_quantity'].sum()

    return i, updated_gold_usage

# 멀티쓰레딩 적용
with ThreadPoolExecutor(max_workers=4) as executor:  # max_workers에 CPU 수 조정 가능
    futures = [executor.submit(update_gold_usage, i) for i in range(len(levelups_ref) - 1)]
    
    # tqdm으로 진행률 표시
    for future in tqdm(as_completed(futures), total=len(futures), desc="레벨업 처리 중"):
        i, usage = future.result()
        levelups_ref.at[i, 'updated_gold_usage'] = usage

# 최종 결과
levelups_ref['final_currency_usage'] = levelups_ref['updated_gold_usage']


In [56]:
check.reset_index(drop=True, inplace=True)

In [59]:
check[20:40]

,player_slug,character_slug,first_session_level,total_purchase,total_ad,highest_level,status,level_from,level_to,event_datetime,death,ad_revival,ad_item,ad_total,purchase,event_datetime_prev,interval,interval_cum,playtime_boss,playtime_dungeon,playtime_tower,playtime_pvp,playtime_farm,playtime_survival,playtime_side,updated_gold_usage,final_currency_usage
20,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,21,22,2021-11-02 22:35:46,0.0,0.0,0.0,0.0,0.0,2021-11-02 22:34:39,0.018889,23.705833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150000,150000
21,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,22,23,2021-11-03 09:20:36,0.0,0.0,0.0,0.0,0.0,2021-11-02 22:35:46,10.747500,34.453333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
22,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,23,24,2021-11-03 09:21:22,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:20:36,0.013056,34.466389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
23,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,24,25,2021-11-03 09:21:54,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:21:22,0.009167,34.475556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
24,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,25,26,2021-11-03 09:22:02,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:21:54,0.002500,34.478056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
25,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,26,27,2021-11-03 09:22:10,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:22:02,0.002500,34.480556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
26,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,27,28,2021-11-03 09:23:04,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:22:10,0.015278,34.495833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
27,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,28,29,2021-11-03 09:24:04,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:23:04,0.016944,34.512778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
28,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,29,30,2021-11-03 09:25:08,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:24:04,0.018056,34.530833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
29,2021-11-01T13:52:32.587Z,샤에,11,19800.0,0.0,79,complete,30,31,2021-11-03 09:25:39,0.0,0.0,0.0,0.0,0.0,2021-11-03 09:25:08,0.008889,34.539722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
